In [ ]:
!pip install -q catboost
!pip install -q workalendar
!pip install -q pycaret
!pip install -q optuna
!pip install -q pycaret[tuners]
!pip uninstall packaging
!pip install packaging

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.4/484.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 43.9 MB/s eta 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as spst
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report,mean_absolute_percentage_error
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.impute import KNNImputer
from workalendar.asia import SouthKorea
cal = SouthKorea()
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
data = pd.read_csv("/content/drive/MyDrive/dacon/train.csv")
test = pd.read_csv("/content/drive/MyDrive/dacon/test.csv")
building_info = pd.read_csv("/content/drive/MyDrive/dacon/building_info.csv")
sample_submission = pd.read_csv("/content/drive/MyDrive/dacon/sample_submission.csv")

In [28]:
def gggg(test,building_info):
    test = test.drop(["num_date_time",'일조(hr)','일사(MJ/m2)'],axis=1)
    test["강수량(mm)"] = test["강수량(mm)"].fillna(0)

    imputer = KNNImputer(n_neighbors=5)
    imputer2 = KNNImputer(n_neighbors=5)
    test[["풍속(m/s)"]] = imputer.fit_transform(test[["풍속(m/s)"]])
    test[["습도(%)"]] = imputer2.fit_transform(test[["습도(%)"]])

    test = pd.merge(test, building_info, how = 'inner', on = '건물번호')
    test = test.drop(['건물번호'],axis=1)

    test['일시1'] = pd.to_datetime(test['일시'])
    test['month'] = test["일시1"].dt.month
    test['day'] = test["일시1"].dt.day
    test['hour'] = test["일시1"].dt.hour
    test['강수 유무'] = np.where(test["강수량(mm)"]==0,0,1)
    test["불쾌지수"] = 9/5*test["기온(C)"]-0.55*(1-(test["습도(%)"]/100))*(9/5*test["기온(C)"]-26)+32

    test["태양광용량(kW)"] = np.where(test["태양광용량(kW)"]=='-',0,test["태양광용량(kW)"])
    test["태양광 유무"] = np.where(test["태양광용량(kW)"]==0,0,1)

    test['측정일시2'] = pd.to_datetime(test['일시1']).dt.date
    test['측정일시2'] = pd.to_datetime(test['측정일시2'])
    test['day_of_week'] = test['측정일시2'].dt.day_name()  # 요일
    test = test.rename(columns={'측정일시2':'Date'})
    holiday=pd.DataFrame(cal.holidays(2022))
    holiday=holiday.rename(columns={0:"Date",1:"holiday"})
    holiday["Date"] = pd.to_datetime(holiday["Date"])
    test = pd.merge(test,holiday,on="Date",how="left")
    test['빨간날'] = np.where((test['day_of_week']=='Sunday') | (test['day_of_week']=='Saturday') | (test['holiday'].isnull()==False),1,0)

    test = test.drop(['holiday','day_of_week','Date','일시'],axis=1)

    test['불쾌지수 단계'] = pd.cut(test['불쾌지수'], bins =[0,68, 75, 80,100] , labels = [0,1,2,3])
    test = pd.get_dummies(test, columns = ['불쾌지수 단계'],drop_first=True)
    # test = test.drop(['연면적(m2)','냉방면적(m2)'],axis=1)
    test["ESS저장용량 유무"] = np.where(test["ESS저장용량(kWh)"]==0,0,1)
    test["PCS저장용량 유무"] = np.where(test["PCS용량(kW)"]==0,0,1)
    test = test.drop(["ESS저장용량(kWh)","PCS용량(kW)"],axis=1)

    test = pd.get_dummies(test,columns=['건물유형'],drop_first=True)
    # test = pd.get_dummies(test,columns=['hour'],drop_first=True)
    # test = pd.get_dummies(test,columns=['month'],drop_first=True)
    # test = pd.get_dummies(test,columns=['day'],drop_first=True)

    return test

def test123(test,building_info):
    test = test.drop(["num_date_time"],axis=1)
    test["강수량(mm)"] = test["강수량(mm)"].fillna(0)

    imputer = KNNImputer(n_neighbors=5)
    imputer2 = KNNImputer(n_neighbors=5)
    test[["풍속(m/s)"]] = imputer.fit_transform(test[["풍속(m/s)"]])
    test[["습도(%)"]] = imputer2.fit_transform(test[["습도(%)"]])

    test = pd.merge(test, building_info, how = 'inner', on = '건물번호')
    test = test.drop(['건물번호'],axis=1)
    test["태양광용량(kW)"] = np.where(test["태양광용량(kW)"]=='-',0,test["태양광용량(kW)"])
    test["태양광 유무"] = np.where(test["태양광용량(kW)"]==0,0,1)
    test['일시1'] = pd.to_datetime(test['일시'])
    test['month'] = test["일시1"].dt.month
    test['day'] = test["일시1"].dt.day
    test['hour'] = test["일시1"].dt.hour
    test['강수 유무'] = np.where(test["강수량(mm)"]==0,0,1)
    test["불쾌지수"] = 9/5*test["기온(C)"]-0.55*(1-(test["습도(%)"]/100))*(9/5*test["기온(C)"]-26)+32

    test['측정일시2'] = pd.to_datetime(test['일시1']).dt.date
    test['측정일시2'] = pd.to_datetime(test['측정일시2'])
    test['day_of_week'] = test['측정일시2'].dt.day_name()  # 요일
    test = test.rename(columns={'측정일시2':'Date'})
    holiday=pd.DataFrame(cal.holidays(2022))
    holiday=holiday.rename(columns={0:"Date",1:"holiday"})
    holiday["Date"] = pd.to_datetime(holiday["Date"])
    test = pd.merge(test,holiday,on="Date",how="left")
    test['빨간날'] = np.where((test['day_of_week']=='Sunday') | (test['day_of_week']=='Saturday') | (test['holiday'].isnull()==False),1,0)

    test["ESS저장용량 유무"] = np.where(test["ESS저장용량(kWh)"]==0,0,1)
    test["PCS저장용량 유무"] = np.where(test["PCS용량(kW)"]==0,0,1)
    test = test.drop(['holiday','day_of_week','Date','일시'],axis=1)


    test['불쾌지수 단계'] = pd.cut(test['불쾌지수'], bins =[0,68, 75, 80,100] , labels = [0,1,2,3])
    test = pd.get_dummies(test, columns = ['불쾌지수 단계'],drop_first=True)
    # test = test.drop(['연면적(m2)','냉방면적(m2)'],axis=1)

    test = test.drop(["ESS저장용량(kWh)","PCS용량(kW)"],axis=1)

    test = pd.get_dummies(test,columns=['건물유형'],drop_first=True)
    # test = pd.get_dummies(test,columns=['hour'],drop_first=True)
    # test = pd.get_dummies(test,columns=['month'],drop_first=True)
    # test = pd.get_dummies(test,columns=['day'],drop_first=True)

    return test

def result(sample_submission,y_pred):
    sample_submission['answer1'] = y_pred
    sample_submission = sample_submission.drop('answer',axis=1)
    sample_submission = sample_submission.rename(columns={'answer1':'answer'})
    sample_submission.to_csv('cat2.csv',index = False)

In [29]:
pd.options.display.max_columns = 50

In [30]:
train = gggg(data,building_info)
train = train.drop('일시1',axis=1)
train

,기온(C),강수량(mm),풍속(m/s),습도(%),전력소비량(kWh),연면적(m2),냉방면적(m2),태양광용량(kW),month,day,hour,강수 유무,불쾌지수,태양광 유무,빨간날,불쾌지수 단계_1,불쾌지수 단계_2,불쾌지수 단계_3,ESS저장용량 유무,PCS저장용량 유무,건물유형_공공,건물유형_대학교,건물유형_데이터센터,건물유형_백화점및아울렛,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_지식산업센터,건물유형_할인마트,건물유형_호텔및리조트
0,18.6,0.0,0.9,42.0,1085.28,110634.00,39570.00,0,6,1,0,0,63.09388,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
1,18.0,0.0,1.1,45.0,1047.36,110634.00,39570.00,0,6,1,1,0,62.46400,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2,17.7,0.0,1.5,45.0,974.88,110634.00,39570.00,0,6,1,2,0,62.08735,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
3,16.7,0.0,1.4,48.0,953.76,110634.00,39570.00,0,6,1,3,0,60.89884,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
4,18.4,0.0,2.8,43.0,986.40,110634.00,39570.00,0,6,1,4,0,62.88788,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,23.1,0.0,0.9,86.0,881.04,57497.84,40035.23,0,8,24,19,0,72.38034,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1
203996,22.4,0.0,1.3,86.0,798.96,57497.84,40035.23,0,8,24,20,0,71.21736,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1
203997,21.3,0.0,1.0,92.0,825.12,57497.84,40035.23,0,8,24,21,0,69.79704,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1
203998,21.0,0.0,0.3,94.0,640.08,57497.84,40035.23,0,8,24,22,0,69.41060,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1


In [31]:
# firstb = train[['건물번호',"불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','태양광 유무','강수 유무','전력소비량(kWh)','빨간날','건물유형','ESS저장용량 유무','PCS저장용량 유무','month','day','hour']]
# x_ = train.drop(['건물번호',"불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','태양광 유무','강수 유무','전력소비량(kWh)','빨간날','건물유형','ESS저장용량 유무','PCS저장용량 유무','month','day','hour'],axis=1)

# scaler = StandardScaler()
# scaler.fit(x_)

# x_train_ = scaler.transform(x_)
# x_train = pd.DataFrame(x_train_,index=x_.index,columns=x_.columns)
# train = pd.concat([x_train,firstb],axis=1)
# train

In [32]:
test = test123(test,building_info)
test = test.drop('일시1',axis=1)
test

,기온(C),강수량(mm),풍속(m/s),습도(%),연면적(m2),냉방면적(m2),태양광용량(kW),태양광 유무,month,day,hour,강수 유무,불쾌지수,빨간날,ESS저장용량 유무,PCS저장용량 유무,불쾌지수 단계_1,불쾌지수 단계_2,불쾌지수 단계_3,건물유형_공공,건물유형_대학교,건물유형_데이터센터,건물유형_백화점및아울렛,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_지식산업센터,건물유형_할인마트,건물유형_호텔및리조트
0,23.5,0.0,2.2,72.0,110634.00,39570.00,0,0,8,25,0,0,71.78980,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,23.0,0.0,0.9,72.0,110634.00,39570.00,0,0,8,25,1,0,71.02840,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,22.7,0.0,1.5,75.0,110634.00,39570.00,0,0,8,25,2,0,70.81675,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,22.1,0.0,1.3,78.0,110634.00,39570.00,0,0,8,25,3,0,70.11262,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,21.8,0.0,1.0,77.0,110634.00,39570.00,0,0,8,25,4,0,69.56514,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,22.5,0.0,0.9,84.0,57497.84,40035.23,0,0,8,31,19,0,71.22400,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
16796,20.7,0.0,0.4,95.0,57497.84,40035.23,0,0,8,31,20,0,68.95035,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
16797,20.2,0.0,0.4,98.0,57497.84,40035.23,0,0,8,31,21,0,68.24604,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
16798,20.1,0.0,1.1,97.0,57497.84,40035.23,0,0,8,31,22,0,68.01203,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [33]:
# firstbbb = test[['건물번호',"불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','태양광 유무','강수 유무','빨간날','건물유형','ESS저장용량 유무','PCS저장용량 유무','month','day','hour']]
# x___ = test.drop(['건물번호',"불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','태양광 유무','강수 유무','빨간날','건물유형','ESS저장용량 유무','PCS저장용량 유무','month','day','hour'],axis=1)

# scaler1 = StandardScaler()
# scaler1.fit(x___)
# x_test = scaler1.transform(x___)
# x_test = pd.DataFrame(x_test,index=x___.index,columns=x___.columns)
# test = pd.concat([x_test,firstbbb],axis=1)
# # test = test.drop(['건물번호','month','day','연면적(m2)','냉방면적(m2)'],axis=1)
# test

In [34]:
from pycaret.regression import *
s = setup(train, target = '전력소비량(kWh)', session_id = 42,fold=5)

,Description,Value
0,Session id,42
1,Target,전력소비량(kWh)
2,Target type,Regression
3,Original data shape,"(204000, 31)"
4,Transformed data shape,"(204000, 31)"
5,Transformed train set shape,"(142800, 31)"
6,Transformed test set shape,"(61200, 31)"
7,Numeric features,29
8,Categorical features,1
9,Preprocess,True


In [36]:
##스케일링하고 라벨인코딩
best = compare_models(n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,94.5023,36031.1095,189.8114,0.9940,0.0947,0.0761,117.5080
et,Extra Trees Regressor,94.2959,35534.8425,188.4685,0.9940,0.0970,0.0771,90.8120
catboost,CatBoost Regressor,156.9287,65594.1030,256.0964,0.9890,0.1460,0.1242,26.5340
dt,Decision Tree Regressor,119.2070,65892.8882,256.5926,0.9890,0.1266,0.0913,2.2400
xgboost,Extreme Gradient Boosting,167.1691,73058.9547,270.2221,0.9878,0.1576,0.1288,23.4560
lightgbm,Light Gradient Boosting Machine,225.6097,109464.3183,330.8254,0.9816,0.1909,0.1728,2.6860
knn,K Neighbors Regressor,263.8764,256128.0969,505.9941,0.9571,0.2135,0.1674,22.4260
gbr,Gradient Boosting Regressor,488.6693,444854.6446,666.9615,0.9254,0.3512,0.3563,22.4900
llar,Lasso Least Angle Regression,936.4901,1764271.5909,1328.2052,0.7044,0.5878,0.6081,0.3200
br,Bayesian Ridge,936.4820,1763952.2170,1328.0848,0.7044,0.5891,0.6073,0.7500


Processing:   0%|          | 0/87 [00:00<?, ?it/s]

In [37]:
rf = create_model('rf',fold=3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,98.4527,39304.4782,198.2536,0.9936,0.0989,0.0548
1,98.9136,38978.6819,197.4302,0.9936,0.1007,0.1287
2,98.8085,38566.0425,196.3824,0.9932,0.0950,0.0537
Mean,98.7249,38949.7342,197.3554,0.9935,0.0982,0.0791
Std,0.1972,302.1592,0.7657,0.0002,0.0024,0.0351


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [38]:
et = create_model('et',fold=3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,98.0713,38316.4530,195.7459,0.9938,0.1001,0.0557
1,99.4002,38237.3077,195.5436,0.9937,0.1036,0.1302
2,99.2833,38999.2977,197.4824,0.9932,0.0980,0.0553
Mean,98.9183,38517.6861,196.2573,0.9935,0.1006,0.0804
Std,0.6008,342.0802,0.8702,0.0003,0.0023,0.0352


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [39]:
dt = create_model('dt',fold=3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,124.0735,72351.4641,268.9823,0.9883,0.1295,0.0674
1,126.6708,74995.0271,273.8522,0.9876,0.1342,0.1585
2,125.6058,71823.9048,267.9998,0.9874,0.1317,0.0672
Mean,125.4500,73056.7987,270.2781,0.9878,0.1318,0.0977
Std,1.0660,1387.3540,2.5589,0.0004,0.0019,0.0430


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
tune_dt = tune_model(dt, search_library = 'optuna',search_algorithm = "tpe",fold=4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,05:49:16
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Decision Tree Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

[I 2023-08-08 05:49:16,631] Searching the best hyperparameters using 142800 samples...


In [ ]:
##스케일링안하고 라벨인코딩

In [ ]:
from pycaret.regression import *
s1 = setup(train, target = '전력소비량(kWh)', session_id = 42,fold=5,use_gpu=True)

,Description,Value
0,Session id,42
1,Target,전력소비량(kWh)
2,Target type,Regression
3,Original data shape,"(204000, 22)"
4,Transformed data shape,"(204000, 22)"
5,Transformed train set shape,"(142800, 22)"
6,Transformed test set shape,"(61200, 22)"
7,Numeric features,20
8,Categorical features,1
9,Preprocess,True


In [ ]:
best = compare_models(n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,90.7071,33713.5397,183.5809,0.9943,0.0934,0.0747,53.5540
rf,Random Forest Regressor,94.0767,35036.5406,187.1631,0.9941,0.0941,0.0772,79.0660
dt,Decision Tree Regressor,118.7109,63513.0189,251.9878,0.9893,0.1258,0.0806,2.0100
xgboost,Extreme Gradient Boosting,170.9523,72356.7516,268.9208,0.9879,0.1681,0.1286,1.1220
catboost,CatBoost Regressor,174.0472,73087.1982,270.3159,0.9878,0.1666,0.1369,6.9520
lightgbm,Light Gradient Boosting Machine,223.4922,106304.0675,326.0239,0.9822,0.1891,0.1725,2.6320
knn,K Neighbors Regressor,263.8828,256137.1125,506.0031,0.9571,0.2135,0.1674,18.9360
gbr,Gradient Boosting Regressor,530.8404,520349.5096,721.3273,0.9128,0.3866,0.3969,22.0140
ridge,Ridge Regression,1007.9120,2339692.2803,1529.4971,0.6080,0.6467,0.7078,0.6160
br,Bayesian Ridge,1007.8825,2339689.9888,1529.4962,0.6080,0.6466,0.7078,0.5540


Processing:   0%|          | 0/87 [00:00<?, ?it/s]

In [ ]:
blender_top3 = blend_models(estimator_list=best)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,91.6137,34621.9763,186.0698,0.9943,0.0939,0.0513
1,92.1709,33724.3702,183.6420,0.9946,0.0943,0.0510
2,92.1306,33145.1691,182.0581,0.9944,0.1006,0.1626
3,92.3915,35405.9807,188.1648,0.9942,0.0908,0.0497
4,92.9360,36462.6028,190.9518,0.9934,0.0904,0.0504
Mean,92.2485,34672.0198,186.1773,0.9942,0.0940,0.0730
Std,0.4280,1181.2454,3.1671,0.0004,0.0036,0.0448


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
final_best = finalize_model(blender_top3)
prediction = predict_model(final_best, data = test)

In [ ]:
prediction

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),태양광 유무,...,hour,강수 유무,불쾌지수,빨간날,ESS저장용량 유무,PCS저장용량 유무,불쾌지수 단계_1,불쾌지수 단계_2,불쾌지수 단계_3,prediction_label
0,1,23.500000,0.0,2.2,72.0,0,110634.000000,39570.000000,0,0,...,0,0,71.789803,0,1,1,1,0,0,2208.054423
1,1,23.000000,0.0,0.9,72.0,0,110634.000000,39570.000000,0,0,...,1,0,71.028397,0,1,1,1,0,0,2208.022395
2,1,22.700001,0.0,1.5,75.0,0,110634.000000,39570.000000,0,0,...,2,0,70.816750,0,1,1,1,0,0,1898.228815
3,1,22.100000,0.0,1.3,78.0,0,110634.000000,39570.000000,0,0,...,3,0,70.112617,0,1,1,1,0,0,1880.548809
4,1,21.799999,0.0,1.0,77.0,0,110634.000000,39570.000000,0,0,...,4,0,69.565140,0,1,1,1,0,0,1880.411207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,22.500000,0.0,0.9,84.0,11,57497.839844,40035.230469,0,0,...,19,0,71.223999,0,1,1,1,0,0,772.912001
16796,100,20.700001,0.0,0.4,95.0,11,57497.839844,40035.230469,0,0,...,20,0,68.950348,0,1,1,1,0,0,711.443611
16797,100,20.200001,0.0,0.4,98.0,11,57497.839844,40035.230469,0,0,...,21,0,68.246040,0,1,1,1,0,0,792.308995
16798,100,20.100000,0.0,1.1,97.0,11,57497.839844,40035.230469,0,0,...,22,0,68.012032,0,1,1,1,0,0,692.897416


In [ ]:
sample_submission['answer']=prediction['prediction_label']
sample_submission

,num_date_time,answer
0,1_20220825 00,2208.054423
1,1_20220825 01,2208.022395
2,1_20220825 02,1898.228815
3,1_20220825 03,1880.548809
4,1_20220825 04,1880.411207
...,...,...
16795,100_20220831 19,772.912001
16796,100_20220831 20,711.443611
16797,100_20220831 21,792.308995
16798,100_20220831 22,692.897416


In [ ]:
sample_submission.to_csv('cat16.csv',index=False)

In [ ]:
##스케일링하고 원핫인코딩